# Читання даних з web на прикладі `clarity-project.info`

In [1]:
# install.packages("rvest")

In [1]:
library(rvest)

In [2]:
url <- "https://clarity-project.info/edr/31978272/finances?current_year=2023"

In [4]:
page <- read_html(url)
page

{html_document}
<html>
[1] <head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8 ...
[2] <body>\n<div class="top-menu-wrap">\n<div class="logo-item-wrap">\n<a hre ...

In [7]:
tables <- html_nodes(page, "table")
tables

{xml_nodeset (10)}
 [1] <table class="table table-relaxed table-md-slim table-bordered w-100 bor ...
 [2] <table class="table table-relaxed table-md-slim table-bordered table-hov ...
 [3] <table class="table table-relaxed table-md-slim table-bordered table-hov ...
 [4] <table class="table table-relaxed table-md-slim table-bordered table-hov ...
 [5] <table class="table table-relaxed table-md-slim table-bordered table-hov ...
 [6] <table class="table table-relaxed table-md-slim table-bordered table-hov ...
 [7] <table class="table table-relaxed table-md-slim table-bordered table-hov ...
 [8] <table class="table table-relaxed table-md-slim table-bordered table-hov ...
 [9] <table class="table table-relaxed table-md-slim table-bordered table-hov ...
[10] <table class="table table-relaxed table-md-slim table-bordered table-hov ...

In [11]:
data <- data.frame(html_table(tables[2]))
colnames(data) <- c("Title", "Code", "Start", "End")
head(data)

,Title,Code,Start,End
,<chr>,<int>,<chr>,<chr>
1,I. Необоротні активи Нематеріальні активи,1000,8 709.00,4 180.00
2,первісна вартість,1001,58 677.00,32 319.00
3,накопичена амортизація,1002,49 968.00,28 139.00
4,Незавершені капітальні інвестиції,1005,48 952.00,27 112.00
5,Основні засоби,1010,293 532.00,363 894.00
6,первісна вартість,1011,777 033.00,896 576.00


In [13]:
str(data)

'data.frame':	51 obs. of  4 variables:
 $ Title: chr  "I. Необоротні активи Нематеріальні активи" "первісна вартість" "накопичена амортизація" "Незавершені капітальні інвестиції" ...
 $ Code : int  1000 1001 1002 1005 1010 1011 1012 1015 1016 1017 ...
 $ Start: chr  "8 709.00" "58 677.00" "49 968.00" "48 952.00" ...
 $ End  : chr  "4 180.00" "32 319.00" "28 139.00" "27 112.00" ...


In [20]:
library(stringr)

data$Start <- as.numeric(str_replace_all(data$Start, " ", ""))
data$End <- as.numeric(str_replace_all(data$End, " ", ""))

head(data)

,Title,Code,Start,End
,<chr>,<int>,<dbl>,<dbl>
1,I. Необоротні активи Нематеріальні активи,1000,8709,4180
2,первісна вартість,1001,58677,32319
3,накопичена амортизація,1002,49968,28139
4,Незавершені капітальні інвестиції,1005,48952,27112
5,Основні засоби,1010,293532,363894
6,первісна вартість,1011,777033,896576


In [22]:
library(openxlsx)

In [26]:
write.xlsx(data, file = "data/zvit.xlsx")

Warning message in file.create(to[okay]):
"cannot create file 'data/zvit.xlsx', reason 'Permission denied'"


## Створення функції

In [27]:
library(rvest)
library(stringr)

# 31978272

In [31]:
read_balance <- function(company_id, year) {
    url <- paste0("https://clarity-project.info/edr/", company_id, "/finances?current_year=", year)
    page <- read_html(url)
    tables <- html_nodes(page, "table")
    data <- data.frame(html_table(tables[2]))
    colnames(data) <- c("Title", "Code", "Start", "End")
    data$Start <- as.numeric(str_replace_all(data$Start, " ", ""))
    data$End <- as.numeric(str_replace_all(data$End, " ", ""))
    return(data)
}

In [33]:
data_2023 <- read_balance(31978272, 2023)
head(data_2023)

,Title,Code,Start,End
,<chr>,<int>,<dbl>,<dbl>
1,I. Необоротні активи Нематеріальні активи,1000,8709,4180
2,первісна вартість,1001,58677,32319
3,накопичена амортизація,1002,49968,28139
4,Незавершені капітальні інвестиції,1005,48952,27112
5,Основні засоби,1010,293532,363894
6,первісна вартість,1011,777033,896576


In [42]:
data_2022 <- read_balance(43457691, 2023)
head(data_2022)

,Title,Code,Start,End
,<chr>,<int>,<dbl>,<dbl>
1,I. Необоротні активи Нематеріальні активи,1000,35.6,18.0
2,первісна вартість,1001,305.5,317.2
3,накопичена амортизація,1002,269.9,299.2
4,Незавершені капітальні інвестиції,1005,102894.2,102912.5
5,Основні засоби:,1010,262.1,0.0
6,первісна вартість,1011,3928.3,2990.2


In [112]:
company_id <- 43457691
years <- c(2020:2022)

In [108]:
read_balance2 <- function(company_id, year = 2023) {
    url <- paste0("https://clarity-project.info/edr/", company_id, "/finances?current_year=", year)
    page <- read_html(url)
    tables <- html_nodes(page, "table")
    data1 <- data.frame(html_table(tables[2]))
    cnames <- c("Title", "Code", paste0("Start", as.character(year)), paste0("End", as.character(year)))
    colnames(data1) <- cnames
    data1[, 3] <- as.numeric(str_replace_all(data1[, 3], " ", ""))
    data1[, 4] <- as.numeric(str_replace_all(data1[, 4], " ", ""))
    return(data1)
}

In [113]:
data <- NULL

for(y in years) {
    df <- read_balance2(company_id, y)
    
    if(is.null(data)) {        
        data <- df
    }
    else {
        data <- cbind(data, df[, 3:4])        
     }

    print(y)
}

data

[1] 2020
[1] 2021
[1] 2022


Title,Code,Start2020,End2020,Start2021,End2021,Start2022,End2022
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
I. Необоротні активи Нематеріальні активи,1000,0,55455,147.8,128.0,128.0,35.6
первісна вартість,1001,0,56214,196.7,295.3,295.3,305.5
накопичена амортизація,1002,0,759,48.9,167.3,167.3,269.9
Незавершені капітальні інвестиції,1005,0,0,57755.2,94334.1,94334.1,102894.2
Основні засоби:,1010,0,0,1428.3,1720.3,1720.3,262.1
первісна вартість,1011,0,0,2131.9,3921.0,3921.0,3928.3
знос,1012,0,0,703.6,2200.7,2200.7,3666.2
Довгострокові біологічні активи,1020,0,0,0.0,NA,0.0,NA
Довгострокові фінансові інвестиції,1030,0,0,0.0,NA,0.0,NA
